# 1. Load the dataset

We will use a benign dataest `Pre_train_D_0.csv`.

In [ ]:
import pandas as pd

def load_dataset(path):
    # parse the dataset
    df = pd.read_csv(path)

    ################ distilled from notebook 0 ################
    # check the integrity
    assert df.isna().any().any() == False, 'There is at least one missing value.'
    assert df['Timestamp'].is_monotonic_increasing, 'Timestamp is not sorted.'

    # type-cast
    df['abstime'] = pd.to_datetime(df['Timestamp'], unit='s').round('us')
    df['monotime'] = df['Timestamp'] - df['Timestamp'].min()
    df['aid_int'] = df['Arbitration_ID'].map(lambda x: int(x, 16))
    df['y'] = df['Class'].map({'Normal': 0, 'Attack': 1})

    return df


df = load_dataset('0_Preliminary/0_Training/Pre_train_D_0.csv')
df


# 2. Data exploration


## Number of Arbitration IDs

In [ ]:
df['Arbitration_ID'].nunique()


## List of Arbitration IDs

In [ ]:
df['Arbitration_ID'].unique()
# sorted(df['Arbitration_ID'].unique())


## Number of CAN messages in each ECU

In [ ]:
df['Arbitration_ID'].value_counts()


260    7466
153    7466
220    7466
251    7465
367    7465
       ... 
5B0      75
043      75
5BE      74
07F      74
57F      37
Name: Arbitration_ID, Length: 73, dtype: int64

## Count the ground-truth

In [ ]:
df['y'].value_counts()


0    179346
Name: y, dtype: int64

## Observation period

In [ ]:
df['Timestamp'].max() - df['Timestamp'].min()


74.65505194664001

## The number of messages per second

$n=\frac{\text{Number of messages}}{\text{Capture period}}$

In [ ]:
n = df.shape[0] / (df['Timestamp'].max() - df['Timestamp'].min())
f'There are {n:.02f} CAN messages per second on average.'


'There are 2402.33 CAN messages per second on average.'

# 3. ECU-wise time interval measurement 

In [ ]:
#TASK 3 
#write a code to add a column to the dataframe that shows the time difference between two consecutive messages


## Group by Arbitration ID

In [ ]:
gb = df.groupby('Arbitration_ID')
gb  # a group-by operator; not a data unless further operaters are given.
gb.size()  # equals to df['Arbitration_ID'].value_counts()

#calculate the time difference between two consecutive messages within the same AID group
df['td'] = gb['Timestamp'].diff() 


In [ ]:
#shows the first 30 rows of the dataframe
df.loc[:30]


In [ ]:
# ECU-wise average time interval
gb['td'].mean()
# gb['Timedelta'].mean().to_dict() # to print entire information


## Summary the CAN dataset

In [ ]:
df_summary = pd.concat([
    gb.size().rename('Count'),
    gb['DLC'].unique(),
    gb['td'].min().rename('Min td'),
    gb['td'].mean().rename('Avg td'),
    gb['td'].max().rename('Max td'),
    gb['td'].std().rename('Std td'),
    gb['Data'].nunique().rename('Num of uniq. payload'),
], axis=1)  # when axis=1, each series is concatenated as a column. axis=0 for the row-concatenation.

pd.options.display.max_rows = 100  # print 100 rows at once
df_summary
